* [reference](https://keras.io/examples/timeseries/timeseries_classification_from_scratch/)

In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [3]:
x_train.shape

(3601, 500)

In [20]:
# data were normalized
np.allclose(x_train.mean(axis=1), 0), np.allclose(x_train.std(axis=1, ddof=1), 1.)

(True, True)

In [26]:
rng = np.random.default_rng(42)
error = rng.normal(size = x_train.shape)

In [32]:
x_train_ = np.dstack((x_train, x_train+error))
x_test_ = np.dstack((x_test, x_test+error[:x_test.shape[0], :x_test.shape[1]]))

In [36]:
x_train.shape[1:]

(500,)

In [45]:
y_train[y_train==-1] = 0
y_test[y_test==-1] = 0

In [39]:
num_classes=2

In [43]:
def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train_.shape[1:], num_classes = num_classes)
# keras.utils.plot_model(model, show_shapes=True)

In [42]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 500, 2)]          0         
                                                                 
 conv1d_7 (Conv1D)           (None, 500, 64)           448       
                                                                 
 batch_normalization_6 (Batc  (None, 500, 64)          256       
 hNormalization)                                                 
                                                                 
 re_lu_6 (ReLU)              (None, 500, 64)           0         
                                                                 
 conv1d_8 (Conv1D)           (None, 500, 64)           12352     
                                                                 
 batch_normalization_7 (Batc  (None, 500, 64)          256       
 hNormalization)                                           

In [46]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)


In [ ]:
history = model.fit(
    x_train_,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [77]:
keras.backend.epsilon()

1e-07

In [78]:
ln.epsilon

0.001